In [5]:
!pip install datasets transformers evaluate
!pip install transformers datasets evaluate scikit-learn


In [1]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [8]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 4. Tokenize text
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/270399 [00:00<?, ? examples/s]

Map:   0%|          | 0/10857 [00:00<?, ? examples/s]

Map:   0%|          | 0/14465 [00:00<?, ? examples/s]

In [2]:
os.environ["WANDB_DISABLED"] = "true"

# 3. Load the multilingual sentiment dataset
dataset = load_dataset("tyqiangz/multilingual-sentiments", "english")

# 4. Preprocess
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# 5. Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 6. Metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# 7. TrainingArguments (very lightweight)
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,   # Only 1 epoch to avoid crash
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",   # No checkpoint saving
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",     # Disable reporting
)

# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"].shuffle(seed=42).select(range(1000)),  # Only 1000 samples
    eval_dataset=encoded_dataset["test"].select(range(500)),                     # Only 500 samples
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 9. Train
trainer.train()

# 10. Evaluate
trainer.evaluate()

# 11. Make a prediction
text = "This product is amazing and exceeded my expectations!"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
prediction = torch.argmax(outputs.logits, dim=-1)

labels = ["negative", "neutral", "positive"]
print(f"Prediction: {labels[prediction]}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0000.parquet:   0%|          | 0.00/156k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/29.6k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/65.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1839 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1839 [00:00<?, ? examples/s]

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Map:   0%|          | 0/870 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [12]:
!pip install -U transformers


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    do_eval=True,
    do_train=True,
    logging_steps=100,
    save_steps=500,
)


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].shuffle(seed=42).select(range(3000)),
    eval_dataset=tokenized_dataset["validation"].select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


<ipython-input-14-0e796fc8e52e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)

In [ ]:
test_text = "I love this product, it's amazing!"
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)
output = model(**inputs)
predicted_class = int(output.logits.argmax(dim=-1))
print(f"Predicted Sentiment: {['Negative', 'Neutral', 'Positive'][predicted_class]}")